In [2]:
import findspark

In [3]:
import re

In [4]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [5]:
import pyspark

In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [7]:
sc = pyspark.SparkContext()

In [8]:
spark = SparkSession \
    .builder \
    .appName("PA3") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [9]:
import pandas as pd
df = pd.read_csv("train.csv")

In [10]:
df = spark.createDataFrame(df)

In [11]:
df1 =df.withColumn('plot',lower(regexp_replace(df['plot'],"[^a-zA-Z ]","")))

In [12]:
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')
df1.take(1)

[nltk_data] Downloading package stopwords to /home/cse587/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[Row(movie_id=23890098, movie_name='Taxi Blues', plot='shlykov a hardworking taxi driver and lyosha a saxophonist develop a bizarre lovehate relationship and despite their prejudices realize they arent so different after all', genre="['World cinema', 'Drama']")]

In [13]:
stop = stopwords.words('english')
df1.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



In [14]:
df1 = df1.dropna()

In [15]:
df2 = df1.select('plot')

In [16]:
rd = df2.rdd.flatMap(list)
data = df2.rdd.map(list)
data = data.map(lambda x: x[0].split())
def dd(x):
    to_remove = []
    for word in x:
        if word not in stop:
            continue
        else:
            to_remove.append(word)
    for word in to_remove:
        x.remove(word)
    return x
data = data.map(dd)
data.take(1)


[['shlykov',
  'hardworking',
  'taxi',
  'driver',
  'lyosha',
  'saxophonist',
  'develop',
  'bizarre',
  'lovehate',
  'relationship',
  'despite',
  'prejudices',
  'realize',
  'arent',
  'different']]

In [17]:
rd1 = rd.flatMap(lambda x: x.split())
rd1 = rd1.filter(lambda x: x not in stop)

In [18]:
m = rd1.map(lambda x: (x,1))

In [19]:
wordCount = m.reduceByKey(lambda a,b: a+b)


In [20]:
common_words = wordCount.filter(lambda x: x[1] > 200)

In [21]:
common_words.count()

4312

In [22]:
features = common_words.keys()

In [23]:
from pyspark .mllib.linalg import SparseVector

In [24]:
from collections import Counter
def remove_uncommon_words(x):
    for word in x:
        if word in features:
            continue
        else:
            x.remove(word)
    return x


In [25]:
features= features.collect()

In [165]:
data = data.map(remove_uncommon_words)

In [27]:
vocab_map = data.flatMap(lambda token: token).distinct() \
    .zipWithIndex().collectAsMap()

In [28]:
v_map = sc.broadcast(vocab_map)
vocab_size = sc.broadcast(len(vocab_map))

In [29]:
tdm = data \
    .map(Counter) \
    .map(lambda counts: {v_map.value[token]: float(counts[token]) for token in counts})\
    .map(lambda index_counts: SparseVector(vocab_size.value, index_counts))

In [30]:
labels = df1.select('genre')

In [31]:
labels= labels.rdd.flatMap(lambda x:x)

["['World cinema', 'Drama']",
 "['Action/Adventure', 'Action', 'Science Fiction', 'Drama']"]

In [32]:
labels.take(3)

["['World cinema', 'Drama']",
 "['Action/Adventure', 'Action', 'Science Fiction', 'Drama']",
 "['Musical', 'Action', 'Drama']"]

In [34]:
sparseVectorsList = tdm.collect()

In [33]:
data.take(1)

[['hardworking',
  'taxi',
  'driver',
  'saxophonist',
  'develop',
  'bizarre',
  'relationship',
  'despite',
  'realize',
  'different']]

In [34]:
movie_ids = df.select('movie_id')
movie_ids = movie_ids.rdd
movie_ids = movie_ids.flatMap(lambda x:x)
movie_ids = movie_ids.collect()


In [103]:
keylist = pd.read_csv('mapping.csv')
keylist.rename(columns={'Unnamed: 0': 'i'}, inplace=True)
keylist = spark.createDataFrame(keylist)
k = keylist.select('0')
k1 = keylist.select('i')
k = k.rdd.flatMap(lambda x:x)
k1 = k1.rdd.flatMap(lambda x:x)
genre_list = k1.zip(k)

In [ ]:
genre_list.collect()

In [177]:
import numpy as np
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel


In [210]:
# def mapFunc(line):
#     val = 0.0
#     vals = line.map(lambda x: (x[0], x[1]))
#     for lab in vals[0]:
#         if label == lab:
#             val = 1.0
#             break
#     return LabeledPoint(val, vals[1])

def test(data):
    ans = []
    for i in range(len(data)):
        val = 0.0
        labels = data[i][0].replace('[','').replace(']','').replace('\'','').split(',')
        for lab in labels:
            if lab == label:
                val = 1.0
                break
        pt = LabeledPoint(val, data[i][1])
        ans.append(pt)
    return ans

def makeClassifier(data):
#     lab2Data = data.map(mapFunc)
    lst = test(data)
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
#     l2dl = lab2Data.collect()
#     return SVMWithSGD.train(sc.parallelize(l2dl), iterations=100)

In [179]:
lab2sparse = labels.zip(tdm)
lab2sparse.take(1)

[("['World cinema', 'Drama']",
  SparseVector(83408, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 41752: 1.0, 41753: 1.0, 41754: 1.0, 41755: 1.0, 41756: 1.0}))]

In [168]:
l2sm = lab2sparse.map(lambda x: (x[0],x[1]))
data = l2sm.collect()

In [206]:
data[0][0]

['World cinema', 'Drama']


['World cinema', ' Drama']

In [211]:
classifiers = []

for label in k.collect():
    print('Trained', label, 'Classifier')
    classifiers.append(makeClassifier(data))
# classifier = makeClassifier(lab2sparse)

Drama
Comedy
Romance Film
Thriller
Action
World cinema
Crime Fiction
Horror
Black-and-white
Indie
Action/Adventure
Adventure
Family Film
Short Film
Romantic drama
Animation
Musical
Science Fiction
Mystery
Romantic comedy


# Part 2: TF-IDF

In [ ]:
 from pyspark.mllib.feature import HashingTF, IDF